In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect2 as w
import scores as s
import ml_window as m
from icecream import ic

import pickle

import plotly.graph_objects as go

In [2]:
a00, a01 = h.import_desired_data("A", "15T")
window_sched = np.array(a01["Window Open"])

In [3]:
with open('../constants/data_combos_230638.pkl', 'rb') as handle:
    test_dict = pickle.load(handle)

In [4]:
for k, v in test_dict.items():
    # print(k)
    obj = m.ML_Window_Detect(v["data"], window_sched)
    obj.run_all()
    test_dict[k]["accuracy"] = obj.accuracy
    test_dict[k]["choices"] = obj.choices

In [5]:
# def show_results():
    

In [6]:
a = pd.Series(test_dict["dt_meas"]["choices"])
a

0      1
1      1
2      0
3      0
4      0
      ..
380    1
381    1
382    1
383    1
384    1
Length: 385, dtype: int64

In [7]:
# need the indices that are true => consider these as "flips"
mask = a.shift() != a
mask

0       True
1      False
2       True
3      False
4      False
       ...  
380     True
381    False
382    False
383    False
384    False
Length: 385, dtype: bool

In [8]:
guesses = a[mask].index
guesses

Int64Index([  0,   2,  20,  23,  25,  26,  29,  30,  31,  32,  33,  38,  39,
             42,  46,  49,  64,  72,  76,  89,  90,  93,  95, 102, 119, 122,
            126, 127, 129, 135, 137, 138, 141, 142, 157, 159, 161, 162, 163,
            185, 186, 190, 195, 196, 203, 204, 206, 207, 209, 223, 233, 237,
            238, 241, 242, 244, 245, 246, 257, 274, 279, 280, 284, 291, 302,
            304, 307, 322, 323, 338, 366, 374, 377, 379, 380],
           dtype='int64')

In [9]:
guess_times = a01["DateTime"][guesses]
guess_times

0     2022-07-20 07:15:00
2     2022-07-20 07:45:00
20    2022-07-20 12:15:00
23    2022-07-20 13:00:00
25    2022-07-20 13:30:00
              ...        
366   2022-07-24 02:45:00
374   2022-07-24 04:45:00
377   2022-07-24 05:30:00
379   2022-07-24 06:00:00
380   2022-07-24 06:15:00
Name: DateTime, Length: 75, dtype: datetime64[ns]

In [10]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=guess_times, y=[1]*len(guess_times),mode='markers',))
# fig.add_trace(go.Scatter(x=guess_times, y=s1.res.apply(lambda x: x[0]*1),mode='markers',))

# fig.add_trace(go.Scatter(x=a01["DateTime"], y=window_sched , mode='lines',))


In [11]:
s1 = s.Scores()
s1.report_scores(guess_times, a01)

,results
hits/guesses,4.0
hits/actions,23.1
(hits + near hits)/guesses,26.7
(hits + near hits)/actions,153.8
misses/guesses,73.3
number of actions,13.0
number of guesses,75.0
hit,3.0
near_hit,17.0
miss,55.0


In [23]:
_dict, _df = h.report_score_ratios_general(guess_times, a01)
_df

,results
hits/guesses,4.0
hits/actions,23.1
(hits + near hits)/guesses,26.7
(hits + near hits)/actions,153.8
misses/guesses,73.3
number of actions,13.0
number of guesses,75.0
hit,3.0
near_hit,17.0
miss,55.0


In [24]:
# what does w.guess_times look like...